# 載入套件

In [1]:
import numpy as np
from scipy.special import comb

# 參數

In [107]:
S = 160     #股價
u = 1.5     #上漲幅度
d = 0.5     #下跌幅度
r = 0.18232 #離散利率
X = 150     #履約價
n = 5      #期數

# 計算R與P

In [108]:
R = round((np.exp(r)),3) # exp(r)=R
P = (R-d)/(u-d)          # P=(R-d)/(u-d)

# 股價以及相對機率

In [109]:
#先建出(n+1)*(n+1)的list，其中+1的原因是為了放入最初股價
Stock_Price = [[0]*(n+1) for i in range(n+1)]
Probibility =  [[0]*(n+1) for i in range(n+1)]
#分層計算股價
#機率部分利用二項式定理計算
for i in range(n+1):
    for j in range(i+1):
        Stock_Price[i][j] = S * u**(i-j) * d**(j) 
        Probibility[i][j] = round(comb(i,j)*P**(i-j) *(1-P)**(j),3)
#將股價以及相對機率一起用print表示
for i in range(0, (n+1)):
    for j in range(i+1):
        print(Stock_Price[i][j], " ", end='')
        print("({})".format(Probibility[i][j]), " ", end='')
    print()

160.0  (1.0)  
240.0  (0.7)  80.0  (0.3)  
360.0  (0.49)  120.0  (0.42)  40.0  (0.09)  
540.0  (0.343)  180.0  (0.441)  60.0  (0.189)  20.0  (0.027)  
810.0  (0.24)  270.0  (0.412)  90.0  (0.265)  30.0  (0.076)  10.0  (0.008)  
1215.0  (0.168)  405.0  (0.36)  135.0  (0.309)  45.0  (0.132)  15.0  (0.028)  5.0  (0.002)  


# 計算買權價格表

In [111]:
#先計算出最後一期的損益
Call_Price =  [[0]*(n+1) for i in range(n+1)]
for j in range(n+1):
    Call_Price[n][j] = max(Stock_Price[n][j] -X,0) 
#先計算出最後一期的損益
#其餘期數利用公式 C_u = (P*C_uu + (1-P)*C_ud)/R
#利用倒推法，故將i進行轉換，使其從最後面開始倒推
for i in range(0,n):
    i = -(i-n+1)
    for j in range(i+1):
        Call_Price[i][j] = round((P*Call_Price[i+1][j] + (1-P)*Call_Price[i+1][j+1])/R,3)
#Hedge Ratio
#h = (C_u-C_d)/(S_u-S_d)
Hedge_Ratio = [[0]*(n+1) for i in range(n)]
for i in range(0,n):
    i = -(i-n+1)
    for j in range(i+1):
        Hedge_Ratio[i][j] = round((Call_Price[i+1][j] - Call_Price[i+1][j+1])/
                                  (Stock_Price[i+1][j] - Stock_Price[i+1][j+1]),5)

In [112]:
#print 出買權價格表以及股數
for i in range(0, (n)):
    for j in range(i+1):
        print(Call_Price[i][j], " ", end='')
        print("({})".format(Hedge_Ratio[i][j]), " ", end='')
    print()
for j in range(n+1):
    print(Call_Price[n][j], " ", end='')

108.842  (0.90254)  
173.932  (0.94108)  29.526  (0.6327)  
276.476  (0.97222)  50.616  (0.72309)  0.0  (0.0)  
436.771  (0.99306)  86.771  (0.82639)  0.0  (0.0)  0.0  (0.0)  
685.0  (1.0)  148.75  (0.94444)  0.0  (0.0)  0.0  (0.0)  0.0  (0.0)  
1065.0  255.0  0  0  0  0  

# 計算賣權價格表

In [115]:
#賣權計算邏輯同上，僅將max函數進行修改
Put_Price =  [[0]*(n+1) for i in range(n+1)]
for j in range(n+1):
    Put_Price[n][j] = max(X-Stock_Price[n][j],0) 
for i in range(0,n):
    i = -(i-n+1)
    for j in range(i+1):
        Put_Price[i][j] = round((P*Put_Price[i+1][j] + (1-P)*Put_Price[i+1][j+1])/R,3)
#B = (uP_d-dP_u)/((u-d)R)
Bond      =  [[0]*(n+1) for i in range(n)]
for i in range(0,n):
    i = -(i-n+1)
    for j in range(i+1):
        Bond[i][j] = round((u*Put_Price[i+1][j+1] - d*Put_Price[i+1][j])/((u-d)*R),5)

In [116]:
#print 出賣權價格表以及債券數量
for i in range(0, (n)):
    for j in range(i+1):
        print(Put_Price[i][j], " ", end='')
        print("({})".format(Bond[i][j]), " ", end='')
    print()
for j in range(n+1):
    print(Put_Price[n][j], " ", end='')

9.124  (24.7175)  
6.27  (20.41)  21.864  (51.24833)  
3.282  (13.28167)  17.422  (50.65125)  46.806  (86.80583)  
0.938  (4.6875)  10.938  (42.1875)  44.167  (104.16667)  84.167  (104.16667)  
0.0  (0.0)  3.75  (18.75)  35.0  (125.0)  95.0  (125.0)  115.0  (125.0)  
0  0  15.0  105.0  135.0  145.0  